# Deep Averageing Netrok Analysis

In [50]:
import torch

embd = torch.nn.Embedding(20, 5)
x = torch.tensor([[1, 2, 3], [0, 2, 4]], dtype=torch.int32)
print(x)
out = embd(x)
print(out)
print(out.shape)
print(out.sum(axis=1))
print(out.sum(axis=1).shape)
lenthes = torch.tensor([3, 2])
print(lenthes.shape)
out.sum(axis=1)/lenthes.reshape([-1, 1])

tensor([[1, 2, 3],
        [0, 2, 4]], dtype=torch.int32)
tensor([[[ 1.0482,  2.0123, -0.5793, -1.6339,  0.5679],
         [-1.3865,  0.5137,  0.2737, -0.3556,  0.5181],
         [-1.4103, -0.2406, -1.9688, -0.3529,  1.5662]],

        [[-0.9058, -1.4235,  0.4452,  0.2892,  1.4724],
         [-1.3865,  0.5137,  0.2737, -0.3556,  0.5181],
         [ 1.4639,  0.1712, -0.5358,  1.1458,  0.2673]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])
tensor([[-1.7486,  2.2854, -2.2744, -2.3424,  2.6523],
        [-0.8284, -0.7387,  0.1831,  1.0793,  2.2579]], grad_fn=<SumBackward1>)
torch.Size([2, 5])
torch.Size([2])


tensor([[-0.5829,  0.7618, -0.7581, -0.7808,  0.8841],
        [-0.4142, -0.3693,  0.0916,  0.5397,  1.1289]], grad_fn=<DivBackward0>)

In [72]:
out = torch.randn(10, 2)
labels = torch.randint(0, 2,size=(10,))
labels
topval, topi = out.topk(1, dim=1)
true_vals = labels[labels == topi.reshape([10,])].shape[0]
error_vals = torch.nonzero(labels-topi.squeeze()).shape[0]
print('accuracy=', true_vals/labels.shape[0])
print('accuracy=', 1-(error_vals/labels.shape[0]))

accuracy= 0.6
accuracy= 0.6


# Glove Emeddings
### Loading Glove Resources
* [main link](https://medium.com/mlearning-ai/load-pre-trained-glove-embeddings-in-torch-nn-embedding-layer-in-under-2-minutes-f5af8f57416a)
* [second link](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76)
* [Download Page](https://nlp.stanford.edu/projects/glove/)

In [188]:
# Unkown token in i_to_word[-1] <unk>
glove_path = '/home/abdullah/Downloads/glove.6B/'
glove_type = 'glove.6B.50d.txt'
file_name = glove_path + glove_type
i_to_word,embeddings = [],[]
words = set()
word_to_i = {}
with open(file_name,'rt') as fi:
    print(full_content[-1])
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    word = full_content[i].split(' ')[0]
    embed = [float(val) for val in full_content[i].split(' ')[1:]]
    i_to_word.append(word)
    word_to_i[word] = i
    words.add(word)
    embeddings.append(embed)

# adding padding token:
kPAD = '<pad>'
i_to_word.append(kPAD)
word_to_i[kPAD] = len(word_to_i)
words.add(kPAD)
embeddings.append(len(embeddings[0]) * [0.0]) 
embeddings[word_to_i[kPAD]][:10]

<unk> 0.072617 -0.51393 0.4728 -0.52202 -0.35534 0.34629 0.23211 0.23096 0.26694 0.41028 0.28031 0.14107 -0.30212 -0.21095 -0.10875 -0.33659 -0.46313 -0.40999 0.32764 0.47401 -0.43449 0.19959 -0.55808 -0.34077 0.078477 0.62823 0.17161 -0.34454 -0.2066 0.1323 -1.8076 -0.38851 0.37654 -0.50422 -0.012446 0.046182 0.70028 -0.010573 -0.83629 -0.24698 0.6888 -0.17986 -0.066569 -0.48044 -0.55946 -0.27594 0.056072 -0.18907 -0.59021 0.55559


400002

In [176]:
# adding pytorch layer
embeddings_tensor = torch.tensor(embeddings)
embed_layer = torch.nn.Embedding.from_pretrained(embeddings_tensor,
                                                padding_idx=word_to_i[kPAD])
print(embed_layer.weight.shape)
print(embed_layer.embedding_dim)

torch.Size([400002, 50])
50


In [174]:
test_w = 'fool'
print(embeddings_tensor[word_to_i[test_w]][:6].numpy())
w_vec =  torch.tensor([ word_to_i[test_w]])
print(embed_layer(w_vec)[0][:6].numpy())

[-0.075706 -0.16436   0.18032  -0.6262    0.52178  -0.14113 ]
[-0.075706 -0.16436   0.18032  -0.6262    0.52178  -0.14113 ]


# Training

## Training with no pretrained Embeddings

In [210]:
!python3  dan.py --num-epochs=100 --checkpoint=10 

Number of Classes= 2
DanModel(
  (embeddings): Embedding(12678, 50, padding_idx=0)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
Saving Model ............
# of steps=10, Avg Train Loss=0.005377, Avg Dev Loss=0.005619, Train Acc=0.962358, Dev Acc=0.646865, Time: 0.341744
Saving Model ............
# of steps=20, Avg Train Loss=0.005284, Avg Dev Loss=0.005535, Train Acc=0.969531, Dev Acc=0.696370, Time: 0.521272
----------------------------------------------------------

start epoch 1
Saving Model ............
# of steps=10, Avg Train Loss=0.005159, Avg Dev Loss=0.005373, Train Acc=0.965909, Dev Acc=0.699670, Time: 0.312261
Saving Model ............
# of steps=

## Traing with glove Embeddings

In [211]:
!python3  dan.py --num-epochs=100 --checkpoint=10 \
    --use-glove --glove-weights='/home/abdullah/Downloads/glove.6B/glove.6B.50d.txt' \
    #--resume --load-model='glove_50d_q_type.pt'  

Loading Glove Embeddings.......
Number of Classes= 2
DanModel(
  (embeddings): Embedding(400002, 50, padding_idx=400001)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
Saving Model ............
# of steps=10, Avg Train Loss=0.005447, Avg Dev Loss=0.005336, Train Acc=0.968040, Dev Acc=0.896040, Time: 0.828138
Saving Model ............
# of steps=20, Avg Train Loss=0.004915, Avg Dev Loss=0.004992, Train Acc=0.995313, Dev Acc=0.924092, Time: 1.119499
----------------------------------------------------------

start epoch 1
Saving Model ............
# of steps=10, Avg Train Loss=0.004598, Avg Dev Loss=0.004428, Train Acc=0.980114, Dev Acc=0.925743, Time: 1.243639

In [201]:
!python3 dan.py  --help

usage: dan.py [-h] [--no-cuda] [--train-file TRAIN_FILE] [--dev-file DEV_FILE]
              [--test-file TEST_FILE] [--batch-size BATCH_SIZE]
              [--num-epochs NUM_EPOCHS] [--grad-clipping GRAD_CLIPPING]
              [--resume] [--test] [--save-model SAVE_MODEL]
              [--load-model LOAD_MODEL] [--limit LIMIT]
              [--checkpoint CHECKPOINT] [--num-workers NUM_WORKERS]
              [--use-glove] [--glove-weights GLOVE_WEIGHTS]

Question Type

options:
  -h, --help            show this help message and exit
  --no-cuda
  --train-file TRAIN_FILE
  --dev-file DEV_FILE
  --test-file TEST_FILE
  --batch-size BATCH_SIZE
  --num-epochs NUM_EPOCHS
  --grad-clipping GRAD_CLIPPING
  --resume
  --test
  --save-model SAVE_MODEL
  --load-model LOAD_MODEL
  --limit LIMIT         Number of training documents
  --checkpoint CHECKPOINT
  --num-workers NUM_WORKERS
                        Number of workers
  --use-glove           Wather to use glove or not
  --glove-weights GL